#Loading dataset

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 12.9 MB/s 
     |████████████████████████████████| 7.6 MB 42.0 MB/s 
     |████████████████████████████████| 182 kB 42.0 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# importing required libraries
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [ ]:
# loading the dataset
df_eval = pd.read_csv('ValidationSet20.csv')
df_eval.head()

,Topic,Text,Question,Answer
0,The Universe,"The Universe is everything we can touch, feel,...",Where is the edge of the Universe?,we cannot see the edge
1,The Universe,"The Universe is everything we can touch, feel,...",What materials are filled in the Space?,Space is also filled with radiation (e.g. ligh...
2,The Universe,"The Universe is everything we can touch, feel,...",What is the speed of light?,"300,000 km per second"
3,Our nearest star,The Sun is our nearest star. The Sun provides ...,Which direction is the Earth spinning?,Earth is spinning towards the east
4,Our nearest star,The Sun is our nearest star. The Sun provides ...,how delayed the sun light to reach earth?,it takes about 8 minutes for sunlight to reach us


In [ ]:
#for understanding dataset characters
from transformers import BertTokenizerFast

#check Error
df = df_eval
answer_start = []
answer_end = []
for i, row in df.iterrows() :
  start = row['Text'].find(row['Answer'])
  if (start==-1):
    print("error", i, row['Answer'])
  answer_start.append(start)
  end = start + len(row['Answer'])
  answer_end.append(end)

print('number of topics', len(df['Topic'].unique()), df['Topic'].unique())

Fast_squad_tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

voca_map = {}
tokens_questions_texts = []

for index, row in df.iterrows():
  input_ids = Fast_squad_tokenizer.encode(row['Question'], row['Text'])
  tokens = Fast_squad_tokenizer.convert_ids_to_tokens(input_ids)
  for token, id in zip(tokens, input_ids):
    #print('{:8}{:8,}'.format(token,id))
    tokens_questions_texts.append(token)

print('len - voca', len(tokens_questions_texts))
print('len - unique voca', len(set(tokens_questions_texts)))
voca_set  = set(tokens_questions_texts)
print('10- unique voca',tokens_questions_texts[:20])

question_n_tokens = []
text_n_tokens = []

for index, row in df.iterrows():
  input_ids = Fast_squad_tokenizer.encode(row['Question'], row['Text'])
  sep_idx = input_ids.index(Fast_squad_tokenizer.sep_token_id)
  question_n_tokens.append(sep_idx+1)
  text_n_tokens.append(len(input_ids) - (sep_idx+1))

print('mean(question_n_tokens)', round(sum(question_n_tokens) / len(question_n_tokens)))
print('mean(text_n_tokens)', round(sum(text_n_tokens) / len(text_n_tokens)))

number of topics 8 ['The Universe' 'Our nearest star' 'The Sun' 'Dwarfs and supergiants'
 'Galaxies' 'What Is Gravity?' 'Mars' 'Birth of the Moon']
len - voca 5570
len - unique voca 699
10- unique voca ['[CLS]', 'where', 'is', 'the', 'edge', 'of', 'the', 'universe', '?', '[SEP]', 'the', 'universe', 'is', 'everything', 'we', 'can', 'touch', ',', 'feel', ',']
mean(question_n_tokens) 11
mean(text_n_tokens) 268


In [ ]:
# Loading the BERT for question answering model which is already finetuned on squad and its corresponding tokenizer
squad_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
import string
import re

def normalize_text(text):
    text = text.lower()
    text = "".join(ch for ch in text if ch not in set(string.punctuation))
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)
    text = " ".join(text.split())
    return text
'''
test = "Wipe ##s out life over much of the planet"
print(normalize_text(test))
'''

'\ntest = "Wipe ##s out life over much of the planet"\nprint(normalize_text(test))\n'

In [ ]:
import torch
import torchvision.models as models

#finetuned_space_squad_model = torch.load('weights_4.pth')
PATH = '/content/drive/MyDrive/BERT-SQuAD/model150.pth'

finetuned_space_squad_model = torch.load(PATH)
finetuned_space_squad_model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [ ]:
after_predicted_answers = []
test_df = df_eval

for i, row in test_df.iterrows():
  question = row['Question']
  text = row['Text']
  input_ids = squad_tokenizer.encode(question, text)
  tokens = squad_tokenizer.convert_ids_to_tokens(input_ids)
  sep_idx = input_ids.index(squad_tokenizer.sep_token_id)
  #print("SEP token index: ", sep_idx)
  num_seg_a = sep_idx + 1
  #print("Number of tokens in segment A (question): ", num_seg_a)
  num_seg_b = len(input_ids) - num_seg_a
  #print("Number of tokens in segment B (text): ", num_seg_b)
  segment_ids = [0]*num_seg_a + [1]*num_seg_b
  #creating the segment ids and making sure every input token has a segment id
  
  output = finetuned_space_squad_model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))
  
  answer_start = torch.argmax(output.start_logits)
  answer_end = torch.argmax(output.end_logits)
  if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
  else:
    print("I am unable to find the answer to this question.")
    answer = " "

  #print("Text:\n{}".format(row["Text"]))
  print("\n *** Question:\n{}".format(question.capitalize()))
  print("\nPredicted Answer:\n{}.".format(answer.capitalize()))
  normal_answer = normalize_text(answer)
  # print("\nNormalized Predicted Answer:\n{}.".format(normal_answer.capitalize()))
  print('\nTruth Answer:\n', row['Answer'])

  after_predicted_answers.append(normal_answer.capitalize())

dict = {'after_predicted_answers': after_predicted_answers}

# Creating Dataframe
df_result = pd.DataFrame(dict)

# Export DataFrame to CSV
df_result.to_csv('df_result_after_predicted_answers.csv')



 *** Question:
Where is the edge of the universe?

Predicted Answer:
We cannot see the edge.

Truth Answer:
 we cannot see the edge

 *** Question:
What materials are filled in the space?

Predicted Answer:
Radiation ( e . g . light and heat ) , magnetic fields and high energy particles ( e . g . cosmic rays ).

Truth Answer:
 Space is also filled with radiation (e.g. light and heat), magnetic fields and high energy particles (e.g. cosmic rays)

 *** Question:
What is the speed of light?

Predicted Answer:
300 , 000 km per second.

Truth Answer:
 300,000 km per second

 *** Question:
Which direction is the earth spinning?

Predicted Answer:
Towards the east.

Truth Answer:
 Earth is spinning towards the east

 *** Question:
How delayed the sun light to reach earth?

Predicted Answer:
It takes about 8 minutes.

Truth Answer:
 it takes about 8 minutes for sunlight to reach us

 *** Question:
How can we study the sun?

Predicted Answer:
Spec ##tro ##graphs.

Truth Answer:
 Special instru

# Comparison before/after fine-tuning

Before

In [ ]:
squad_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
before_predicted_answers = []

for i, row in test_df.iterrows():
  question = row['Question']
  text = row['Text']
  input_ids = squad_tokenizer.encode(question, text)
  tokens = squad_tokenizer.convert_ids_to_tokens(input_ids)
  sep_idx = input_ids.index(squad_tokenizer.sep_token_id)
  #print("SEP token index: ", sep_idx)
  num_seg_a = sep_idx + 1
  #print("Number of tokens in segment A (question): ", num_seg_a)
  num_seg_b = len(input_ids) - num_seg_a
  #print("Number of tokens in segment B (text): ", num_seg_b)
  segment_ids = [0]*num_seg_a + [1]*num_seg_b
  #creating the segment ids and making sure every input token has a segment id
  
  output = squad_model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))
  
  answer_start = torch.argmax(output.start_logits)
  answer_end = torch.argmax(output.end_logits)
  if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
  else:
    answer = " "
    print("----- I am unable to find the answer to this question.")

  #print("Text:\n{}".format(row["Text"]))
  print("\n###Question:\n{}".format(question.capitalize()))
  print("\nPredicted Answer:\n{}.".format(answer.capitalize()))
  normal_answer = normalize_text(answer)
  print("\nNormalized Predicted Answer:\n{}.".format(normal_answer.capitalize()))
  print('\nTruth Answer:\n', row['Answer'])

  before_predicted_answers.append(normal_answer.capitalize())

dict = {'before_predicted_answers': before_predicted_answers}

# Creating Dataframe
df_result = pd.DataFrame(dict)

# Export DataFrame to CSV
df_result.to_csv('df_result_before_predicted_answers.csv')



###Question:
Where is the edge of the universe?

Predicted Answer:
We cannot see the edge.

Normalized Predicted Answer:
We cannot see edge.

Truth Answer:
 we cannot see the edge

###Question:
What materials are filled in the space?

Predicted Answer:
Radiation ( e . g . light and heat ) , magnetic fields and high energy particles ( e . g . cosmic rays ).

Normalized Predicted Answer:
Radiation e g light and heat magnetic fields and high energy particles e g cosmic rays.

Truth Answer:
 Space is also filled with radiation (e.g. light and heat), magnetic fields and high energy particles (e.g. cosmic rays)

###Question:
What is the speed of light?

Predicted Answer:
300 , 000 km per second.

Normalized Predicted Answer:
300 000 km per second.

Truth Answer:
 300,000 km per second

###Question:
Which direction is the earth spinning?

Predicted Answer:
East.

Normalized Predicted Answer:
East.

Truth Answer:
 Earth is spinning towards the east

###Question:
How delayed the sun light to r

In [ ]:
import pandas as pd

report_before = pd.read_csv('df_result_before_predicted_answers.csv')
report_after = pd.read_csv('df_result_after_predicted_answers.csv')

df_combine = pd.concat([report_before,report_after, df_eval['Answer']], axis=1)
df_combine.drop(columns='Unnamed: 0')

df_combine.head()

,Unnamed: 0,before_predicted_answers,Unnamed: 0,after_predicted_answers,Answer
0,0,We cannot see edge,0,We cannot see edge,we cannot see the edge
1,1,Radiation e g light and heat magnetic fields a...,1,Radiation e g light and heat magnetic fields a...,Space is also filled with radiation (e.g. ligh...
2,2,300 000 km per second,2,300 000 km per second,"300,000 km per second"
3,3,East,3,Towards east,Earth is spinning towards the east
4,4,8 minutes,4,It takes about 8 minutes,it takes about 8 minutes for sunlight to reach us


In [ ]:
df_combine = df_combine.drop(columns='Unnamed: 0')
df_combine.head()
df_combine.to_csv('df_combine.csv')


#EVAL

## BLEU

In [ ]:
# loading the dataset
import pandas as pd

df_result = pd.read_csv('/content/yoonjungchoi_BERT_prediction.csv')
df_result.head()

,Unnamed: 0,before_predicted_answers,after_predicted_answers,Answer
0,0,We cannot see edge,We cannot see edge,we cannot see the edge
1,1,Radiation e g light and heat magnetic fields a...,Radiation e g light and heat magnetic fields a...,Space is also filled with radiation (e.g. ligh...
2,2,300 000 km per second,300 000 km per second,"300,000 km per second"
3,3,East,Towards east,Earth is spinning towards the east
4,4,8 minutes,It takes about 8 minutes,it takes about 8 minutes for sunlight to reach us


In [ ]:
gt_answer = df_result['Answer'].values.tolist()
gt_answer
gt_answer = list(map(lambda x: x.lower(), gt_answer))
print(gt_answer)

after_answer = df_result['after_predicted_answers'].values.tolist()
after_answer = list(map(lambda x: x.lower(), after_answer))
print(after_answer)

['we cannot see the edge', 'space is also filled with radiation (e.g. light and heat), magnetic fields and high energy particles (e.g. cosmic rays)', '300,000 km per second', 'earth is spinning towards the east', 'it takes about 8 minutes for sunlight to reach us', 'special instruments known as spectrographs tell us that the sun', 'this finds its way to the surface as visible, infrared and ultraviolet light, and x-rays', 'some stars are much bigger and hotter than the sun', 'the red supergiant antares is 800 times wider than the sun.', 'blue-white stars', 'the largest galaxies look like squashed balls', 'they contain up to 10 million million stars', 'galaxies were born only a few hundred million years after the universe was created', 'every object in the universe attracts every other object in the universe', 'the strength of gravity depends on the size and density of an object', "hanks to visiting spacecraft, we know that the planet's appearance is due to rust in the martian rocks.", '

In [ ]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 118 kB 12.8 MB/s 


In [ ]:
with open("file_gt", 'w') as output:
    for row in gt_answer:
        output.write(str(row) + '\n')

In [ ]:
with open("file_pred", 'w') as output:
    for row in after_answer:
        output.write(str(row) + '\n')

In [ ]:
!sacrebleu file_gt < file_pred

{
 "name": "BLEU",
 "score": 20.2,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "60.8/43.8/35.2/24.4 (BP = 0.519 ratio = 0.604 hyp_len = 148 ref_len = 245)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


In [ ]:
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 115 kB 24.7 MB/s 
     |████████████████████████████████| 451 kB 58.4 MB/s 
     |████████████████████████████████| 212 kB 64.2 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 127 kB 62.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 5.7 MB/s 
     |████████████████████████████████| 5.5 MB 41.2 MB/s 
     |████████████████████████████████| 7.6 MB 36.7 MB/s 


## BERTScore

In [ ]:
df_result['after_predicted_answers'].fillna('', inplace=True)

from evaluate import load
bertscore = load("bertscore")
predictions = df_result['after_predicted_answers'].values.tolist()
references = df_result['Answer'].values.tolist()
results = bertscore.compute(predictions=predictions, references=references, lang="en")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [ ]:
print(results)
print(sum(results['f1']) / len(results['f1']))

{'precision': [0.957879900932312, 0.898545503616333, 0.9804800152778625, 0.8637048006057739, 0.9273241758346558, 0.8127007484436035, 0.931794285774231, 0.9846529364585876, 0.8111888766288757, 0.9465488791465759, 0.8421710133552551, 0.9468986392021179, 0.9669249653816223, 0.8380249738693237, 0.9332160353660583, 0.9110209941864014, 0.7758268713951111, 0.8995574712753296, 0.8896724581718445, 0.9475516080856323], 'recall': [0.9542105197906494, 0.8895824551582336, 0.9638479948043823, 0.8969682455062866, 0.8996131420135498, 0.8070738911628723, 0.8949445486068726, 0.9823423027992249, 0.8299635052680969, 0.9237885475158691, 0.8457452058792114, 0.9225039482116699, 0.9543734788894653, 0.8594260215759277, 0.9001162648200989, 0.8587921857833862, 0.8285013437271118, 0.8278716206550598, 0.9543976783752441, 0.8873205184936523], 'f1': [0.956041693687439, 0.8940415382385254, 0.9720928072929382, 0.8800222873687744, 0.9132584929466248, 0.8098775744438171, 0.9129977226257324, 0.9834963083267212, 0.8204687

## F1 tokens

In [ ]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [ ]:
df_result.head()

,Unnamed: 0,before_predicted_answers,after_predicted_answers,Answer
0,0,We cannot see edge,We cannot see edge,we cannot see the edge
1,1,Radiation e g light and heat magnetic fields a...,Radiation e g light and heat magnetic fields a...,Space is also filled with radiation (e.g. ligh...
2,2,300 000 km per second,300 000 km per second,"300,000 km per second"
3,3,East,Towards east,Earth is spinning towards the east
4,4,8 minutes,It takes about 8 minutes,it takes about 8 minutes for sunlight to reach us


In [ ]:
f1_list = []

for i, row in df_result.iterrows() :
  pred = row['after_predicted_answers']
  gt = row['Answer']
  f1_list.append(compute_f1(pred, gt))

f1_list
print(sum(f1_list) / len(f1_list))

0.35825152909787444
